In [ ]:
# Configuración e importación
import pandas as pd
import requests
import json
import time
import sqlite3
from typing import List, Dict, Any, Tuple
import logging
from datetime import datetime, timedelta
import os
import importlib

# Cargar la Api de deepseek
try:
    import config
    importlib.reload(config)  # Forzar recarga
    DEEPSEEK_API_KEY = config.DEEPSEEK_API_KEY
    DB_PATH = config.DB_PATH
    print("✅ Configuración cargada desde config.py (recargado)")
except ImportError as e:
    print(f"❌ Error: {e}")
    # Poner las credenciales directamente
    DEEPSEEK_API_KEY = "tu_api_key_real_aqui"
    DB_PATH = "tfm_database.db"
    print("🔄 Usando configuración directa")

# Configuración DeepSeek API
DEEPSEEK_API_URL = "https://api.deepseek.com/v1/chat/completions"

# Configurar logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

print(f"\n✅ Configuración lista")
print(f"📁 DB Path: {DB_PATH}")
print(f"🔑 API Key: {'✅ CONFIGURADA' if DEEPSEEK_API_KEY and DEEPSEEK_API_KEY != 'tu_api_key_real_aqui' else '❌ PENDIENTE'}")

✅ Configuración cargada desde config.py (recargado)

✅ Configuración lista
📁 DB Path: tfm_database.db
🔑 API Key: ✅ CONFIGURADA


In [ ]:
# Clase GeneradorOportunidades
class GeneradorOportunidades:
    def __init__(self, api_key: str, db_path: str):
        self.api_key = api_key
        self.db_path = db_path
        self.headers = {
            "Authorization": f"Bearer {api_key}",
            "Content-Type": "application/json"
        }
        self.api_url = DEEPSEEK_API_URL
        
        # Factores de queja que nos interesan
        self.factores_interes = [
            "precio_elevado",
            "calidad_comida_mala", 
            "presentacion_mala",
            "variedad_escasa"
        ]
    
    def get_connection(self):
        return sqlite3.connect(self.db_path)
    
    def obtener_fecha_limite(self):
        """Obtiene la fecha límite (hace 1 año)"""
        return (datetime.now() - timedelta(days=365)).strftime('%Y-%m-%d')
    
    def obtener_restaurantes_con_quejas(self):
        """Obtiene restaurantes que tienen quejas de interés en el último año"""
        fecha_limite = self.obtener_fecha_limite()
        
        query = """
        SELECT DISTINCT r.restaurante_id, res.tripadvisor_id, res.nombre
        FROM Reviews r
        JOIN Restaurantes res ON r.restaurante_id = res.id
        WHERE r.factores_queja IS NOT NULL 
        AND r.fecha_publicacion >= ?
        """
        
        conn = self.get_connection()
        try:
            restaurantes = pd.read_sql(query, conn, params=(fecha_limite,))
            print(f"📊 Restaurantes con reviews en último año: {len(restaurantes)}")
            return restaurantes
        except Exception as e:
            print(f"❌ Error obteniendo restaurantes: {e}")
            return pd.DataFrame()
        finally:
            conn.close()
    
    def obtener_reviews_restaurante(self, tripadvisor_id: int):
        """Obtiene las reviews de un restaurante con quejas de interés en el último año"""
        fecha_limite = self.obtener_fecha_limite()
        
        query = """
        SELECT r.review_id, r.titulo_es, r.contenido_es, r.factores_queja, r.rating, r.fecha_publicacion
        FROM Reviews r
        JOIN Restaurantes res ON r.restaurante_id = res.id
        WHERE res.tripadvisor_id = ? 
        AND r.fecha_publicacion >= ?
        AND r.factores_queja IS NOT NULL
        ORDER BY r.fecha_publicacion DESC
        """
        
        conn = self.get_connection()
        try:
            reviews = pd.read_sql(query, conn, params=(tripadvisor_id, fecha_limite))
            
            # Filtrar solo reviews con factores de interés
            reviews_con_interes = []
            for _, review in reviews.iterrows():
                try:
                    factores = json.loads(review['factores_queja'])
                    # Verificar si tiene al menos un factor de interés
                    if any(factores.get(factor, 0) == 1 for factor in self.factores_interes):
                        reviews_con_interes.append(review)
                except:
                    continue
            
            return pd.DataFrame(reviews_con_interes)
            
        except Exception as e:
            print(f"❌ Error obteniendo reviews: {e}")
            return pd.DataFrame()
        finally:
            conn.close()
    
    def verificar_oportunidad_existente(self, tripadvisor_id: int, reviews_ids: List[str]) -> bool:
        """Verifica si ya existe una oportunidad para este conjunto de reviews"""
        conn = self.get_connection()
        try:
            reviews_json = json.dumps(sorted(reviews_ids))
            query = "SELECT COUNT(*) FROM Oportunidades WHERE tripadvisor_id = ? AND reviews_utilizadas = ?"
            count = conn.execute(query, (tripadvisor_id, reviews_json)).fetchone()[0]
            return count > 0
        finally:
            conn.close()
    
    def llamar_api_deepseek(self, prompt: str, max_tokens: int = 3000) -> str:
        """Llama a la API de DeepSeek"""
        payload = {
            "model": "deepseek-chat",
            "messages": [{"role": "user", "content": prompt}],
            "temperature": 0.1,
            "max_tokens": max_tokens
        }
        
        try:
            response = requests.post(self.api_url, headers=self.headers, json=payload, timeout=60)
            response.raise_for_status()
            result = response.json()
            return result['choices'][0]['message']['content'].strip()
        except Exception as e:
            logger.error(f"❌ Error API: {e}")
            raise
    
    def generar_resumen_problematicas(self, reviews_df: pd.DataFrame, nombre_restaurante: str) -> str:
        """Genera un resumen de las problemáticas detectadas"""
        
        # Preparar texto con todas las reviews
        reviews_texto = ""
        for _, review in reviews_df.iterrows():
            reviews_texto += f"REVIEW {review['review_id']} (Rating: {review['rating']}/5):\n"
            reviews_texto += f"Título: {review['titulo_es']}\n"
            reviews_texto += f"Contenido: {review['contenido_es']}\n"
            
            # Añadir factores de queja específicos
            try:
                factores = json.loads(review['factores_queja'])
                factores_activos = [f for f in self.factores_interes if factores.get(f, 0) == 1]
                if factores_activos:
                    reviews_texto += f"Factores problema: {', '.join(factores_activos)}\n"
            except:
                pass
            reviews_texto += "\n" + "-"*50 + "\n"
        
        prompt = f"""
        Eres un analista comercial especializado en el sector HORECA. 
        Analiza las siguientes críticas del restaurante "{nombre_restaurante}" y genera un resumen objetivo de las problemáticas principales.
        
        CRÍTICAS DEL RESTAURANTE:
        {reviews_texto}
        
        INSTRUCCIONES:
        1. Resume las problemáticas principales SIN inventar información
        2. Sé específico y basado únicamente en las críticas
        3. Agrupa problemas similares
        4. Mantén un tono profesional y objetivo
        5. Máximo 300 palabras
        
        RESUMEN DE PROBLEMÁTICAS:
        """
        
        return self.llamar_api_deepseek(prompt)
    
    def generar_oportunidades_comerciales(self, problematicas: str, nombre_restaurante: str) -> str:
        """Genera oportunidades comerciales basadas en las problemáticas"""
        
        prompt = f"""
        Eres un comercial experto en el sector HORECA. 
        Basándote en las siguientes problemáticas del restaurante "{nombre_restaurante}", genera oportunidades comerciales concretas.
        
        PROBLEMÁTICAS DETECTADAS:
        {problematicas}
        
        INSTRUCCIONES ESPECÍFICAS:
        Para CADA problemática identificada, propón 2 oportunidades comerciales:
        - Oportunidad 1: Solución directa al problema mencionado
        - Oportunidad 2: Producto/servicio complementario que prevenga el problema
        
        EJEMPLO:
        Problema: "El pollo tenía mal sabor"
        Oportunidades:
        1. Ofertar gama premium de carnes de ave con certificado de calidad
        2. Proponer formación en técnicas de cocción y conservación de carnes
        
        FORMATO DE RESPUESTA:
        Para cada problemática, usar:
        **Problemática X**: [descripción breve]
        - Oportunidad 1: [solución concreta]
        - Oportunidad 2: [solución complementaria]
        
        Sé específico con productos/servicios que podrían solucionar cada problema.
        """
        
        return self.llamar_api_deepseek(prompt, max_tokens=4000)
    
    def procesar_restaurante(self, tripadvisor_id: int, nombre_restaurante: str) -> bool:
        """Procesa un restaurante y genera oportunidades si aplica"""
        print(f"\n🔍 Analizando restaurante: {nombre_restaurante} (ID: {tripadvisor_id})")
        
        # 1. Obtener reviews relevantes
        reviews_df = self.obtener_reviews_restaurante(tripadvisor_id)
        if reviews_df.empty:
            print("   ⏭️  No hay reviews con factores de interés")
            return False
        
        print(f"   📊 Reviews con quejas de interés: {len(reviews_df)}")
        
        # 2. Verificar si ya existe oportunidad para estas reviews
        reviews_ids = reviews_df['review_id'].tolist()
        if self.verificar_oportunidad_existente(tripadvisor_id, reviews_ids):
            print("   ⏭️  Ya existe oportunidad para estas reviews")
            return False
        
        # 3. Generar resumen de problemáticas
        print("   🤖 Generando resumen de problemáticas...")
        problematicas = self.generar_resumen_problematicas(reviews_df, nombre_restaurante)
        
        # 4. Generar oportunidades comerciales
        print("   💡 Generando oportunidades comerciales...")
        oportunidades = self.generar_oportunidades_comerciales(problematicas, nombre_restaurante)
        
        # 5. Guardar en base de datos
        conn = self.get_connection()
        try:
            insert_query = """
            INSERT INTO Oportunidades 
            (tripadvisor_id, reviews_utilizadas, problematicas_detectadas, oportunidades)
            VALUES (?, ?, ?, ?)
            """
            reviews_json = json.dumps(sorted(reviews_ids))
            conn.execute(insert_query, (tripadvisor_id, reviews_json, problematicas, oportunidades))
            conn.commit()
            print("   ✅ Oportunidad guardada en la tabla Oportunidades de SQLite")
            return True
            
        except Exception as e:
            print(f"   ❌ Error guardando oportunidad: {e}")
            conn.rollback()
            return False
        finally:
            conn.close()
    
    def procesar_todos_restaurantes(self, delay: int = 5):
        """Procesa todos los restaurantes con quejas"""
        print("🚀 INICIANDO GENERACIÓN DE OPORTUNIDADES")
        print("=" * 60)
        
        # 1. Obtener restaurantes a procesar
        restaurantes_df = self.obtener_restaurantes_con_quejas()
        if restaurantes_df.empty:
            print("❌ No hay restaurantes para procesar")
            return
        
        print(f"📊 Restaurantes a procesar: {len(restaurantes_df)}")
        
        # 2. Procesar cada restaurante
        procesados = 0
        creados = 0
        errores = 0
        
        for _, restaurante in restaurantes_df.iterrows():
            try:
                procesados += 1
                print(f"\n[{procesados}/{len(restaurantes_df)}] ", end="")
                
                exito = self.procesar_restaurante(
                    restaurante['tripadvisor_id'], 
                    restaurante['nombre']
                )
                
                if exito:
                    creados += 1
                else:
                    print("   ⏭️  Saltado (sin reviews relevantes o ya existe)")
                
                # Delay para evitar rate limiting
                if procesados < len(restaurantes_df):
                    print(f"   ⏳ Esperando {delay} segundos...")
                    time.sleep(delay)
                    
            except Exception as e:
                errores += 1
                print(f"   ❌ Error procesando restaurante: {e}")
                continue
        
        # 3. Mostrar resumen
        print(f"\n🎉 PROCESAMIENTO COMPLETADO")
        print("=" * 60)
        print(f"📊 RESUMEN:")
        print(f"   Restaurantes procesados: {procesados}")
        print(f"   Oportunidades creadas: {creados}")
        print(f"   Errores: {errores}")

# Inicializar generador
generador = GeneradorOportunidades(DEEPSEEK_API_KEY, DB_PATH)
print("✅ Generador de oportunidades inicializado")

✅ Generador de oportunidades inicializado


In [ ]:
# Verificación inicial
print("🔍 VERIFICACIÓN INICIAL")

# 1. Test de conexión API
print("\n🧪 TESTEANDO CONEXIÓN API...")
try:
    test_prompt = "Responde con 'OK' si la conexión funciona."
    respuesta = generador.llamar_api_deepseek(test_prompt, max_tokens=10)
    print(f"✅ API: {respuesta}")
except Exception as e:
    print(f"❌ Error API: {e}")

# 2. Verificar datos disponibles
print("\n📊 DATOS DISPONIBLES:")
with generador.get_connection() as conn:
    # Total restaurantes
    total_rest = conn.execute("SELECT COUNT(*) FROM Restaurantes").fetchone()[0]
    print(f"   Restaurantes en BD: {total_rest}")
    
    # Reviews con factores de interés
    fecha_limite = generador.obtener_fecha_limite()
    query = """
    SELECT COUNT(*) 
    FROM Reviews r
    WHERE r.factores_queja IS NOT NULL 
    AND r.fecha_publicacion >= ?
    """
    reviews_relevantes = conn.execute(query, (fecha_limite,)).fetchone()[0]
    print(f"   Reviews relevantes (último año): {reviews_relevantes}")
    
    # Oportunidades existentes
    try:
        oportunidades_existentes = conn.execute("SELECT COUNT(*) FROM Oportunidades").fetchone()[0]
        print(f"   Oportunidades existentes: {oportunidades_existentes}")
    except:
        print("   Oportunidades existentes: 0 (tabla no existe aún)")

# 3. Mostrar factores de interés
print(f"\n🎯 FACTORES DE INTERÉS:")
for factor in generador.factores_interes:
    print(f"   - {factor}")

🔍 VERIFICACIÓN INICIAL

🧪 TESTEANDO CONEXIÓN API...
✅ API: OK

📊 DATOS DISPONIBLES:
   Restaurantes en BD: 40
   Reviews relevantes (último año): 36
   Oportunidades existentes: 10

🎯 FACTORES DE INTERÉS:
   - precio_elevado
   - calidad_comida_mala
   - presentacion_mala
   - variedad_escasa


In [ ]:
# Procesamiento completo
print("🚀 INICIANDO GENERACIÓN DE OPORTUNIDADES COMPLETA")

# Procesar todos los restaurantes
generador.procesar_todos_restaurantes(delay=5)

🚀 INICIANDO GENERACIÓN DE OPORTUNIDADES COMPLETA
🚀 INICIANDO GENERACIÓN DE OPORTUNIDADES
📊 Restaurantes con reviews en último año: 13
📊 Restaurantes a procesar: 13

[1/13] 
🔍 Analizando restaurante: Octopus Restaurant - Pompeu Fabra (ID: 23990713)
   ⏭️  No hay reviews con factores de interés
   ⏭️  Saltado (sin reviews relevantes o ya existe)
   ⏳ Esperando 5 segundos...

[2/13] 
🔍 Analizando restaurante: Restaurant A4MANS (ID: 33219282)
   📊 Reviews con quejas de interés: 1
   ⏭️  Ya existe oportunidad para estas reviews
   ⏭️  Saltado (sin reviews relevantes o ya existe)
   ⏳ Esperando 5 segundos...

[3/13] 
🔍 Analizando restaurante: CROAK'S Girona (ID: 12506471)
   📊 Reviews con quejas de interés: 3
   ⏭️  Ya existe oportunidad para estas reviews
   ⏭️  Saltado (sin reviews relevantes o ya existe)
   ⏳ Esperando 5 segundos...

[4/13] 
🔍 Analizando restaurante: Originem (ID: 23687398)
   📊 Reviews con quejas de interés: 3
   ⏭️  Ya existe oportunidad para estas reviews
   ⏭️  Saltad

In [5]:
# Verificación de resultados en la tabla Oportunidades
print("🔍 VERIFICANDO RESULTADOS EN TABLA OPORTUNIDADES")

with generador.get_connection() as conn:
    # Estadísticas finales
    total_oportunidades = conn.execute("SELECT COUNT(*) FROM Oportunidades").fetchone()[0]
    print(f"📊 Total oportunidades en tabla: {total_oportunidades}")
    
    if total_oportunidades > 0:
        # Mostrar estructura de la tabla
        print(f"\n🏗️  ESTRUCTURA DE LA TABLA:")
        columnas = conn.execute("PRAGMA table_info(Oportunidades)").fetchall()
        for col in columnas:
            print(f"   - {col[1]} ({col[2]})")
        
        # Mostrar oportunidades creadas
        print(f"\n📝 OPORTUNIDADES CREADAS:")
        oportunidades = conn.execute("""
        SELECT o.id, r.nombre, o.tripadvisor_id, o.fecha_insercion,
               json_array_length(o.reviews_utilizadas) as num_reviews
        FROM Oportunidades o
        JOIN Restaurantes r ON o.tripadvisor_id = r.tripadvisor_id
        ORDER BY o.fecha_insercion DESC
        LIMIT 10
        """).fetchall()
        
        for op in oportunidades:
            print(f"\n--- ID: {op[0]} ---")
            print(f"   Restaurante: {op[1]}")
            print(f"   Tripadvisor ID: {op[2]}")
            print(f"   Fecha: {op[3]}")
            print(f"   Reviews utilizadas: {op[4]}")
        
        # Mostrar ejemplo completo de una oportunidad
        print(f"\n📄 EJEMPLO COMPLETO (primera oportunidad):")
        ejemplo = conn.execute("""
        SELECT r.nombre, o.problematicas_detectadas, o.oportunidades
        FROM Oportunidades o
        JOIN Restaurantes r ON o.tripadvisor_id = r.tripadvisor_id
        ORDER BY o.id
        LIMIT 1
        """).fetchone()
        
        if ejemplo:
            nombre, problematicas, oportunidades = ejemplo
            print(f"\n🏪 RESTAURANTE: {nombre}")
            print(f"\n🔴 PROBLEMÁTICAS (primeras 300 caracteres):")
            print(problematicas[:300] + "..." if len(problematicas) > 300 else problematicas)
            print(f"\n🟢 OPORTUNIDADES (primeras 300 caracteres):")
            print(oportunidades[:300] + "..." if len(oportunidades) > 300 else oportunidades)
    else:
        print("ℹ️  No hay oportunidades en la tabla")

🔍 VERIFICANDO RESULTADOS EN TABLA OPORTUNIDADES
📊 Total oportunidades en tabla: 10

🏗️  ESTRUCTURA DE LA TABLA:
   - id (INTEGER)
   - tripadvisor_id (INTEGER)
   - reviews_utilizadas (TEXT)
   - problematicas_detectadas (TEXT)
   - oportunidades (TEXT)
   - fecha_insercion (TIMESTAMP)

📝 OPORTUNIDADES CREADAS:

--- ID: 10 ---
   Restaurante: Formaticum
   Tripadvisor ID: 24077637
   Fecha: 2025-10-18 18:53:23
   Reviews utilizadas: 4

--- ID: 9 ---
   Restaurante: La Pedra
   Tripadvisor ID: 2715456
   Fecha: 2025-10-18 18:52:41
   Reviews utilizadas: 1

--- ID: 8 ---
   Restaurante: Da Vinci Girona
   Tripadvisor ID: 23177354
   Fecha: 2025-10-18 18:52:08
   Reviews utilizadas: 1

--- ID: 7 ---
   Restaurante: Jardins Gala
   Tripadvisor ID: 27533952
   Fecha: 2025-10-18 18:51:40
   Reviews utilizadas: 3

--- ID: 6 ---
   Restaurante: Restaurant Gran Muralla I
   Tripadvisor ID: 1154276
   Fecha: 2025-10-18 18:51:04
   Reviews utilizadas: 1

--- ID: 5 ---
   Restaurante: Indigo Resta